In [2]:
import pandas as pd

In [37]:
train_df = pd.read_csv('data/train.csv', sep=";", na_values="?")
test_df = pd.read_csv('data/test.csv', sep=";", na_values="?")

train_df.head()

,loan_id,account_id,date,amount,duration,payments,status,district_id,frequency,client_id,...,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,credit total,withdrawal total,First trans date,household (avg)
0,5314,1787,930705,96396,12,8033,-1,30,weekly issuance,2166,...,9650,3.38,3.67,100,2985.0,2804,20100.0,NaN,930322,NaN
1,5316,1801,930711,165960,36,4610,1,46,monthly issuance,2181,...,8369,1.79,2.31,117,2854.0,2618,229893.7,177684.8,930213,4167.0
2,6863,9188,930728,127080,60,2118,1,45,monthly issuance,11314,...,8390,2.28,2.89,132,2080.0,2122,75146.0,54873.0,930208,7936.0
3,5325,1843,930803,105804,36,2939,1,12,monthly issuance,2235,...,8754,3.83,4.31,137,3804.0,3868,120309.8,86017.2,930130,6994.0
4,7240,11013,930906,274740,60,4579,1,1,weekly issuance,13539,...,12541,0.29,0.43,167,85677.0,99107,276327.1,235214.0,930214,756.0


In [48]:
# NaN values

def household_NaN(df: pd.DataFrame):
    df.loc[df['household (avg)'].isna(), 'household (avg)'] = 0
    return df

    

0       0.0
1    4167.0
2    7936.0
3    6994.0
4     756.0
Name: household (avg), dtype: float64

In [38]:
# Data Treatment
def define_final_balance(df: pd.DataFrame):
    df['final balance'] = df.apply(lambda x: x['credit total'] - x['withdrawal total'], axis=1)
    return df

In [39]:
define_final_balance(train_df)

,loan_id,account_id,date,amount,duration,payments,status,district_id,frequency,client_id,...,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,credit total,withdrawal total,First trans date,household (avg),final balance
0,5314,1787,930705,96396,12,8033,-1,30,weekly issuance,2166,...,3.38,3.67,100,2985.0,2804,20100.0,NaN,930322,NaN,NaN
1,5316,1801,930711,165960,36,4610,1,46,monthly issuance,2181,...,1.79,2.31,117,2854.0,2618,229893.7,177684.8,930213,4167.0,52208.9
2,6863,9188,930728,127080,60,2118,1,45,monthly issuance,11314,...,2.28,2.89,132,2080.0,2122,75146.0,54873.0,930208,7936.0,20273.0
3,5325,1843,930803,105804,36,2939,1,12,monthly issuance,2235,...,3.83,4.31,137,3804.0,3868,120309.8,86017.2,930130,6994.0,34292.6
4,7240,11013,930906,274740,60,4579,1,1,weekly issuance,13539,...,0.29,0.43,167,85677.0,99107,276327.1,235214.0,930214,756.0,41113.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,6818,9030,961212,155616,48,3242,1,72,monthly issuance,11122,...,3.80,4.79,110,9672.0,9208,451400.8,390706.8,950121,2587.0,60694.0
399,5625,3189,961215,222180,60,3703,-1,29,monthly issuance,3855,...,2.82,3.60,113,818.0,888,375691.6,316112.6,951129,NaN,59579.0
400,6805,8972,961221,45024,48,938,1,70,monthly issuance,11050,...,6.63,7.75,81,9878.0,10108,170898.6,132514.2,960521,4719.0,38384.4
401,7233,10963,961225,115812,36,3217,1,16,monthly issuance,13480,...,1.12,1.54,107,1874.0,1913,844304.3,802426.2,950520,NaN,41878.1


In [53]:
df = train_df[train_df["unemploymant rate '95"].isna()]
df[df["unemploymant rate '96"].isna()]

,loan_id,account_id,date,amount,duration,payments,status,district_id,frequency,client_id,...,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,credit total,withdrawal total,First trans date,household (avg),final balance
